In [1]:
from pprint import pprint
import json

import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.documents as documents
import azure.cosmos.errors as errors

In [2]:
COSMOSDB_ENDPOINT = '{AZURE_COSMOS_DB_ENDPOINT}'
COSMOSDB_PRIMARY_KEY = '{AZURE_COSMOS_DB_PRIMARY_KEY}'
COSMOSDB_DB_ID = 'FinancialDatabase'
COSMOSDB_COLL_ID = 'InvestorCollection'

client = cosmos_client.CosmosClient(COSMOSDB_ENDPOINT, { 'masterKey': COSMOSDB_PRIMARY_KEY })
database_link = f'dbs/{COSMOSDB_DB_ID}'
partition_key = '/company'
throughput = 500
collection_link = f'{database_link}/colls/{COSMOSDB_COLL_ID}'

In [3]:
def create_database(id):
    print('Create database')

    try:
        client.CreateDatabase({"id": id})
        print(f'Database with id "{id}" created')

    except errors.HTTPFailure as e:
        if e.status_code == 409:
           print(f'A database with id "{id}" already exists')
        else: 
            raise

In [4]:
create_database(COSMOSDB_DB_ID)

Create database
Database with id "FinancialDatabase" created


In [5]:
def create_container(id, throughput, partition_key):
    try:
        coll = {
            "id": id,
            "partitionKey": {
                "paths": [
                    partition_key
                ],
                "kind": "Hash",
                "version": 2
            }
        }
        collection_options = { 'offerThroughput': throughput }
        collection = client.CreateContainer(database_link, coll, collection_options)

        print(f'Collection with id "{id}" created')
        print(f'Partition Key - "{partition_key}"')

    except errors.CosmosError as e:
        if e.status_code == 409:
            print(f'A collection with id "{id}" already exists')
        else:
            raise

In [6]:
create_container(COSMOSDB_COLL_ID, throughput, partition_key)

Collection with id "InvestorCollection" created
Partition Key - "/company"


In [7]:
# This simple stored procedure will echo the input parameter string with the
# text Hello as a prefix.

sproc_definition = {
    'id': 'greetCaller',
    'serverScript': '''
        function greetCaller(name) {
             var context = getContext();
             var response = context.getResponse();
             response.setBody("Hello " + name);
         }
    '''
}

sproc = client.CreateStoredProcedure(collection_link, sproc_definition)

In [8]:
# The output should be "Hello Person".

sproc_result = client.ExecuteStoredProcedure(sproc['_self'], ['Person'], {'partitionKey':'example'})

pprint(sproc_result)

'Hello Person'


In [9]:
# Inside the JavaScript callback, users can either handle the exception or
# throw an error. In case a callback is not provided and there is an error,
# the Azure Cosmos DB runtime throws an error. This stored procedures creates
# a new document and uses a nested callback function to return the document
# as the body of the response.

sproc_definition = {
    'id': 'createDocument',
    'serverScript': '''
         function createDocument(doc) {
             var context = getContext();
             var collection = context.getCollection();
             var accepted = collection.createDocument(
                 collection.getSelfLink(),
                 doc,
                 function (err, newDoc) {
                     if (err) throw new Error('Error' + err.message);
                     context.getResponse().setBody(newDoc);
                 }
             );
             if (!accepted) return;
         }
    '''
}

sproc = client.CreateStoredProcedure(collection_link, sproc_definition)

In [10]:
# You should see a new document in your collection. Azure Cosmos DB has
# assigned additional fields to the document such as id and _etag.

sproc_result = client.ExecuteStoredProcedure(sproc['_self'],
                                             { "company": "contosoairlines", "industry": "travel" },
                                             {'partitionKey':'contosoairlines'})

pprint(sproc_result)

{'_attachments': 'attachments/',
 '_etag': '"00001f00-0000-0100-0000-5e347bd10000"',
 '_rid': 'qOpgAPYZ494BAAAAAAAAAA==',
 '_self': 'dbs/qOpgAA==/colls/qOpgAPYZ494=/docs/qOpgAPYZ494BAAAAAAAAAA==/',
 'company': 'contosoairlines',
 'id': '37de2b14-3722-fe79-5bc6-348a1d2a2b30',
 'industry': 'travel'}


In [11]:
def query_documents(collection_link, query, options={ "enableCrossPartitionQuery": True }):
    try:
        results = list(client.QueryItems(collection_link, query, options))
        return results

    except errors.HTTPFailure as e:
        if e.status_code == 404:
            print("Document doesn't exist")
        elif e.status_code == 400:
            # Can occur when we are trying to query on excluded paths
            print("Bad Request exception occured: ", e)
            pass
        else:
            raise

    finally:
        print()

In [12]:
# This query will retrieve the document you have just created.

query = 'SELECT * FROM investors WHERE investors.company = "contosoairlines" AND investors.industry = "travel"'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 1
{'_attachments': 'attachments/',
 '_etag': '"00001f00-0000-0100-0000-5e347bd10000"',
 '_rid': 'qOpgAPYZ494BAAAAAAAAAA==',
 '_self': 'dbs/qOpgAA==/colls/qOpgAPYZ494=/docs/qOpgAPYZ494BAAAAAAAAAA==/',
 '_ts': 1580497873,
 'company': 'contosoairlines',
 'id': '37de2b14-3722-fe79-5bc6-348a1d2a2b30',
 'industry': 'travel'}


In [13]:
# This stored procedure will use the console.log feature that’s normally used
# in browser-based JavaScript to write output to the console. In the context
# of Azure Cosmos DB, this feature can be used to capture diagnostics logging
# information that can be returned after the stored procedure is executed.

sproc_definition = {
    'id': 'createDocumentWithLogging',
    'serverScript': '''
         function createDocumentWithLogging(doc) {
             console.log("procedural-start ");
             var context = getContext();
             var collection = context.getCollection();
             console.log("metadata-retrieved ");
             var accepted = collection.createDocument(
                 collection.getSelfLink(),
                 doc,
                 function (err, newDoc) {
                     console.log("callback-started ");
                     if (err) throw new Error('Error' + err.message);
                     context.getResponse().setBody(newDoc.id);
                 }
             );
             console.log("async-doc-creation-started ");
             if (!accepted) return;
             console.log("procedural-end");
         }
    '''
}

sproc = client.CreateStoredProcedure(collection_link, sproc_definition)

In [14]:
# You should see the unique id of a new document in your collection. The
# console.log is visible when using Data Explorer in Azure Cosmos DB portal.

sproc_result = client.ExecuteStoredProcedure(sproc['_self'],
                                             { "company": "contosoairlines" },
                                             {'partitionKey':'contosoairlines'})

pprint(sproc_result)

'3319edb4-4e5d-0fd2-7613-29c474de57fa'


In [15]:
# This is the same stored procedure as you created previously but it is using
# a named function instead of an implicit callback function inline.

sproc_definition = {
    'id': 'createDocumentWithFunction',
    'serverScript': '''
         function createDocumentWithFunction(document) {
             var context = getContext();
             var collection = context.getCollection();
             if (!collection.createDocument(collection.getSelfLink(), document, documentCreated))
                 return;
             function documentCreated(error, newDocument) {
                 if (error) throw new Error('Error' + error.message);
                 context.getResponse().setBody(newDocument);
             }
         }
    '''
}

sproc = client.CreateStoredProcedure(collection_link, sproc_definition)

In [16]:
# Notice that the stored procedure execution has failed. Stored procedures
# are bound to a specific partition key. In this example, tried to execute
# the stored procedure within the context of the adventureworks partition key.
# Within the stored procedure, we tried to create a new document using the
# contosoairlines partition key. The stored procedure was unable to create a
# new document (or access existing documents) in a partition key other than
# the one specified when the stored procedure is executed. This caused the
# stored procedure to fail. You are not able to create or manipulate documents
# across partition keys within a stored procedure.

try:
    sproc_result = client.ExecuteStoredProcedure(sproc['_self'],
                                                 { "company": "contosoairlines" },
                                                 {'partitionKey':'adventureworks'})
except errors.HTTPFailure as e:
    pprint(e)

HTTPFailure('Status code: 400\n{"code":"BadRequest","message":"Message: {\\"Errors\\":[\\"Encountered exception while executing function. Exception = Error: Error{\\\\\\"Errors\\\\\\":[\\\\\\"Requests originating from scripts cannot reference partition keys other than the one for which client request was submitted.\\\\\\"]}\\\\r\\\\nStack trace: Error: Error{\\\\\\"Errors\\\\\\":[\\\\\\"Requests originating from scripts cannot reference partition keys other than the one for which client request was submitted.\\\\\\"]}\\\\n   at documentCreated (script.js:8:29)\\\\n   at Anonymous function (script.js:686:29)\\"]}\\r\\nActivityId: bca40562-d83f-4806-a228-8aa8331a7462, Request URI: /apps/c21e6828-0313-4658-9618-353217af2fe6/services/3f1e6ba2-0ac7-493c-9a9e-e7e676989849/partitions/d02939dd-2f96-4ab0-afc5-26d70994eb24/replicas/132249702016963621p/, RequestStats: \\r\\nRequestStartTime: 2020-01-31T19:11:14.2416575Z, RequestEndTime: 2020-01-31T19:11:14.2416575Z,  Number of regions attempted:1

In [17]:
# You should see a new document in your collection. Azure Cosmos DB has
# assigned additional fields to the document such as id and _etag.

sproc_result = client.ExecuteStoredProcedure(sproc['_self'],
                                             { "company": "adventureworks" },
                                             {'partitionKey':'adventureworks'})

pprint(sproc_result)

{'_attachments': 'attachments/',
 '_etag': '"00002400-0000-0100-0000-5e347bd20000"',
 '_rid': 'qOpgAPYZ494DAAAAAAAAAA==',
 '_self': 'dbs/qOpgAA==/colls/qOpgAPYZ494=/docs/qOpgAPYZ494DAAAAAAAAAA==/',
 'company': 'adventureworks',
 'id': 'b50940ff-86cd-3eac-27eb-2c10a8af2c1c'}


In [18]:
# This query will retrieve the document you have just created.

query = 'SELECT * FROM investors WHERE investors.company = "adventureworks"'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 1
{'_attachments': 'attachments/',
 '_etag': '"00002400-0000-0100-0000-5e347bd20000"',
 '_rid': 'qOpgAPYZ494DAAAAAAAAAA==',
 '_self': 'dbs/qOpgAA==/colls/qOpgAPYZ494=/docs/qOpgAPYZ494DAAAAAAAAAA==/',
 '_ts': 1580497874,
 'company': 'adventureworks',
 'id': 'b50940ff-86cd-3eac-27eb-2c10a8af2c1c'}


In [19]:
# This stored procedure uses nested callbacks to create two seperate
# documents. You may have scenarios where your data is split across multiple
# JSON documents and you will need to add or modify multiple documents in a
# single stored procedure.


sproc_definition = {
    'id': 'createTwoDocuments',
    'serverScript': '''
         function createTwoDocuments(companyName, industry, taxRate) {
             var context = getContext();
             var collection = context.getCollection();
             var firstDocument = {
                 company: companyName,
                 industry: industry
             };
             var secondDocument = {
                 company: companyName,
                 tax: {
                     exempt: false,
                     rate: taxRate
                 }
             };
             var firstAccepted = collection.createDocument(collection.getSelfLink(), firstDocument, 
                 function (firstError, newFirstDocument) {
                     if (firstError) throw new Error('Error' + firstError.message);
                     var secondAccepted = collection.createDocument(collection.getSelfLink(), secondDocument,
                         function (secondError, newSecondDocument) {
                             if (secondError) throw new Error('Error' + secondError.message);      
                             context.getResponse().setBody({
                                 companyRecord: newFirstDocument,
                                 taxRecord: newSecondDocument
                             });
                         }
                     );
                     if (!secondAccepted) return;    
                 }
             );
             if (!firstAccepted) return;    
         }
    '''
}

create_two_doc_sproc = client.CreateStoredProcedure(collection_link, sproc_definition)

In [20]:
# You should see a new document in your collection. Azure Cosmos DB has
# assigned additional fields to the document such as id and _etag.

sproc_result = client.ExecuteStoredProcedure(create_two_doc_sproc['_self'],
                                             ['abcairways', 'travel', '1.05'],
                                             {'partitionKey':'abcairways'})

pprint(sproc_result)

{'companyRecord': {'_attachments': 'attachments/',
                   '_etag': '"00002600-0000-0100-0000-5e347bd20000"',
                   '_rid': 'qOpgAPYZ494EAAAAAAAAAA==',
                   '_self': 'dbs/qOpgAA==/colls/qOpgAPYZ494=/docs/qOpgAPYZ494EAAAAAAAAAA==/',
                   'company': 'abcairways',
                   'id': '87e38da6-09fc-954f-a10c-586d15260486',
                   'industry': 'travel'},
 'taxRecord': {'_attachments': 'attachments/',
               '_etag': '"00002700-0000-0100-0000-5e347bd20000"',
               '_rid': 'qOpgAPYZ494FAAAAAAAAAA==',
               '_self': 'dbs/qOpgAA==/colls/qOpgAPYZ494=/docs/qOpgAPYZ494FAAAAAAAAAA==/',
               'company': 'abcairways',
               'id': 'f36b247c-614b-02fd-34b0-902e4debcf21',
               'tax': {'exempt': False, 'rate': '1.05'}}}


In [21]:
# Transactions are deeply and natively integrated into Cosmos DB’s JavaScript
# programming model. Inside a JavaScript function, all operations are
# automatically wrapped under a single transaction. If the JavaScript
# completes without any exception, the operations to the database are
# committed. We are going to change the stored procedure to put in a different
# company name for the second document. This should cause the stored procedure
# to fail since the second document uses a different partition key. If there
# is any exception that’s propagated from the script, Cosmos DB’s JavaScript
# runtime will roll back the whole transaction. This will effectively ensure
# that the first or second documents are not commited to the database.

sproc_definition = {
    'id': 'createTwoDocuments',
    'body': '''
         function createTwoDocuments(companyName, industry, taxRate) {
             var context = getContext();
             var collection = context.getCollection();
             var firstDocument = {
                 company: companyName,
                 industry: industry
             };
             var secondDocument = {
                 company: companyName + "_taxprofile",
                 tax: {
                     exempt: false,
                     rate: taxRate
                 }
             };
             var firstAccepted = collection.createDocument(collection.getSelfLink(), firstDocument, 
                 function (firstError, newFirstDocument) {
                     if (firstError) throw new Error('Error' + firstError.message);
                     console.log('Created: ' + newFirstDocument.id);
                     var secondAccepted = collection.createDocument(collection.getSelfLink(), secondDocument,
                         function (secondError, newSecondDocument) {
                             if (secondError) throw new Error('Error' + secondError.message); 
                             console.log('Created: ' + newSecondDocument.id);                   
                             context.getResponse().setBody({
                                 companyRecord: newFirstDocument,
                                 taxRecord: newSecondDocument
                             });
                         }
                     );
                     if (!secondAccepted) return;    
                 }
             );
             if (!firstAccepted) return;    
         }
    '''
}

sproc = client.ReplaceStoredProcedure(create_two_doc_sproc['_self'], sproc_definition)

In [22]:
# This stored procedure failed to create the second document so the entire
# transaction was rolled back.

try:
    sproc_result = client.ExecuteStoredProcedure(sproc['_self'],
                                                 ['jetsonairways', 'travel', '1.15'],
                                                 {'partitionKey':'jetsonairways'})

except errors.HTTPFailure as e:
    pprint(e)

HTTPFailure('Status code: 400\n{"code":"BadRequest","message":"Message: {\\"Errors\\":[\\"Encountered exception while executing function. Exception = Error: Error{\\\\\\"Errors\\\\\\":[\\\\\\"Requests originating from scripts cannot reference partition keys other than the one for which client request was submitted.\\\\\\"]}\\\\r\\\\nStack trace: Error: Error{\\\\\\"Errors\\\\\\":[\\\\\\"Requests originating from scripts cannot reference partition keys other than the one for which client request was submitted.\\\\\\"]}\\\\n   at Anonymous function (script.js:22:47)\\\\n   at Anonymous function (script.js:686:29)\\"]}\\r\\nActivityId: 62f95e7d-6046-4a27-8986-31cf5c21d463, Request URI: /apps/c21e6828-0313-4658-9618-353217af2fe6/services/3f1e6ba2-0ac7-493c-9a9e-e7e676989849/partitions/d02939dd-2f96-4ab0-afc5-26d70994eb24/replicas/132249702016963621p/, RequestStats: \\r\\nRequestStartTime: 2020-01-31T19:11:14.7816709Z, RequestEndTime: 2020-01-31T19:11:14.7916626Z,  Number of regions attempt

In [23]:
# This query won’t retrieve any documents since the transaction was rolled back.

query = 'SELECT * FROM investors WHERE investors.company = "jetsonairways"'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')


Length of query results: 0


In [24]:
def delete_database(id):
    print('Delete Database')

    try:
       database_link = 'dbs/' + id
       client.DeleteDatabase(database_link)

       print(f'Database with id "{id}" was deleted')

    except errors.HTTPFailure as e:
        if e.status_code == 404:
           print('A database with id "{id}" does not exist')
        else: 
            raise

In [25]:
# Clean up
delete_database(COSMOSDB_DB_ID)

Delete Database
Database with id "FinancialDatabase" was deleted
